In [ ]:
# encoding=utf8

# Feature Engineering with Pipeline Talk at ODSC West 2016, Santa Clara



In [69]:
import pandas as pd
import numpy as np

## Load Titanic Data

In [70]:
df = pd.read_csv("../../DataSets/pybk021master/titanic/train.csv")

In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [72]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [73]:
# Select Features / Target

In [74]:
features = ["Sex", "Age", "Fare", "Pclass", "SibSp",  "Parch", "Embarked"]
X = df[features].copy()
y = df["Survived"]

In [75]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
Sex         891 non-null object
Age         714 non-null float64
Fare        891 non-null float64
Pclass      891 non-null int64
SibSp       891 non-null int64
Parch       891 non-null int64
Embarked    889 non-null object
dtypes: float64(2), int64(3), object(2)
memory usage: 48.8+ KB


In [76]:
## To Emphasise One hot encoding, I am going to fill the missing values of Embarked with Frequently used values
## Also since only to values of Embarked are missing, I am going to fill it with Most common Embarked Station

In [77]:
X.Embarked.value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [78]:
X["Embarked"] = X["Embarked"].fillna("S")

In [79]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
Sex         891 non-null object
Age         714 non-null float64
Fare        891 non-null float64
Pclass      891 non-null int64
SibSp       891 non-null int64
Parch       891 non-null int64
Embarked    891 non-null object
dtypes: float64(2), int64(3), object(2)
memory usage: 48.8+ KB


In [80]:
X.head(2)

,Sex,Age,Fare,Pclass,SibSp,Parch,Embarked
0,male,22.0,7.2500,3,1,0,S
1,female,38.0,71.2833,1,1,0,C


## Pre-Processing Modules in Scikit-learn

In [81]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, LabelBinarizer, Imputer, FunctionTransformer
from sklearn.preprocessing import  OneHotEncoder, Imputer, StandardScaler, LabelBinarizer, LabelEncoder

from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion, make_union

In [82]:
from sklearn.linear_model import LogisticRegression

In [83]:
## Make a Pipeline of 

In [84]:
## Encode Values - Pclass, Sex and Embarked

In [85]:
# One Hot Encode Pclass

class_encoder = OneHotEncoder(sparse=False)
class_encoder.fit_transform(X[["Pclass"]])[:5]

array([[ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.]])

In [86]:
# One Hot Encoding doesn't work on Text Data
gender_encoder = OneHotEncoder(sparse=False)
gender_encoder.fit_transform(X[["Embarked"]])[:5]

ValueError: could not convert string to float: 'Q'

### Unfortunately One Hot Encoder Doesn't work with Categorical Values :-(

In [87]:
### Let's try Label Encoder and Label Binarizer

In [88]:
gender_encoder = LabelEncoder()
gender_encoder.fit_transform(X["Sex"])[:5]

array([1, 0, 0, 0, 1])

In [89]:
## Let's do the same for Embarked Station

embarked_encoder = LabelEncoder()
embarked_values = embarked_encoder.fit_transform(X["Embarked"])[:5]
embarked_values

array([2, 0, 2, 2, 2])

In [90]:
## We will then need to pass those values to One Hot Encoder to encode into binary values

### Label Encoding and One Hot Encoding in one step via LabelBinarizer

In [91]:
embarked_encoder = LabelBinarizer()
embarked_encoder.fit_transform(X["Embarked"])[:5]

array([[0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]])

### OK.  That looks good.  Let's put these together in a Pipeline

In [92]:
make_pipeline(LabelEncoder(), LogisticRegression()).fit(X[["Sex"]], y)

TypeError: fit_transform() takes 2 positional arguments but 3 were given

### Label Encoders Don't work with Pipelines :-(

## Why?

### Github Issues:
    * https://github.com/scikit-learn/scikit-learn/pull/7327
    * https://github.com/scikit-learn/scikit-learn/pull/7375
    
**Note**: Looks like they are looking to fix OneHotEncoding to take Categorical Values in Version 0.19 instead of fixing LabelEcoders to work with pipelines

### For now, We need to go back to Pandas and encode Gender and Embarked to Numbers before using OneHotEncoder

In [93]:
# encode string features as numbers
X['Sex'] = X['Sex'].map({"male": 1, "female": 0})
X['Embarked'] = X['Embarked'].replace({'S': 1, 'C': 2, 'Q': 3})

X.head(5)

,Sex,Age,Fare,Pclass,SibSp,Parch,Embarked
0,1,22.0,7.2500,3,1,0,1
1,0,38.0,71.2833,1,1,0,2
2,0,26.0,7.9250,3,0,0,1
3,0,35.0,53.1000,1,1,0,1
4,1,35.0,8.0500,3,0,0,1


### Now we can put them all together in a Pipeline :-)

In [94]:
Pipeline([
        ("features", OneHotEncoder()),   # One Hot Encode Values
        ("model", LogisticRegression())  # Model
        ]).fit(X[["Pclass", "Sex", "Embarked"]], y)

Pipeline(steps=[('features', OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)), ('model', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

## Let's add other Features as well using FeatureUnion and make one Pipeline

In [95]:
from column_selector import ColumnSelector, ColumnImputer

In [96]:
from sklearn.model_selection import train_test_split

## Also Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [97]:
feature_mapper = make_union(
    make_pipeline(ColumnSelector(["Pclass", "Sex", "Embarked"]), OneHotEncoder()),
    make_pipeline(ColumnSelector(["Age", "Fare"]), Imputer(), StandardScaler()),  # Mean Zero
    ColumnSelector(["SibSp", "Parch"])
)

# Put the combined Features into a Pipeline
model_pipe = Pipeline([
        ("features", feature_mapper),   # Combined Features
        ("model", LogisticRegression())  # Model
        ]).fit(X_train, y_train)

print("Accuracy Score on Test Data: {:.3f}".format(model_pipe.score(X_test, y_test)))

Accuracy Score on Test Data: 0.807


## Hurray - We Put it all together in a Pipeline

#### But can we do better and not have to manually convert Labels into Numbers before passing into OneHotEncoder?

## Enter Sklearn-Pandas

In [98]:
! pip install sklearn-pandas

You are using pip version 9.0.1, however version 9.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [99]:
from sklearn_pandas import DataFrameMapper

import warnings
warnings.filterwarnings("ignore")

In [100]:
features = ["Sex", "Age", "Fare", "Pclass", "SibSp",  "Parch", "Embarked"]
X = df[features].copy()
y = df["Survived"]

# Train / Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y)

X.head(2)

,Sex,Age,Fare,Pclass,SibSp,Parch,Embarked
0,male,22.0,7.2500,3,1,0,S
1,female,38.0,71.2833,1,1,0,C


In [101]:
mapper = DataFrameMapper([
                (["Pclass"], OneHotEncoder()),
                (["Sex"], LabelEncoder()),
                (["Age", "Fare"], [Imputer(strategy="mean"), StandardScaler()]),
                ("Embarked", [ColumnImputer(strategy="most_frequent"), LabelBinarizer()]),
                (["SibSp", "Parch"], None)  # No Transformations necessary
            ])

pipeline = Pipeline(steps=[
        ("features", mapper),
        ("model", LogisticRegression())
    ])

pipeline.fit(X_train, y_train)

print("Accuracy Score on Test Data: {:.3f}".format(pipeline.score(X_test, y_test)))

Accuracy Score on Test Data: 0.785


### Awesome.  We were able to use LibaryBinarizer and Encoder without having to convert to Numbers

## Feature Engineering 

* Polynomial Features
* Dimensionality Reduction
* Feature Selection

In [102]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectFromModel

In [103]:
pipeline = Pipeline(steps=[
        ("features", mapper),
        ("selection", PCA(n_components=0.95)),
        ("model", LogisticRegression())
    ])

pipeline.fit(X_train, y_train)

print("Accuracy Score on Test Data: {:.3f}".format(pipeline.score(X_test, y_test)))

Accuracy Score on Test Data: 0.789


## Put them all Together and Grid Search for Hyper Parameter Tuning

In [104]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV

In [105]:
pipeline = Pipeline(steps=[
        ("features", mapper),
        ("rf_model", RandomForestClassifier())
    ])

params = {
    "rf_model__n_estimators": [10, 100, 200],
    "rf_model__min_samples_leaf": [1, 3, 5],
    "rf_model__max_depth": [None, 10, 7, 5, 3]
}

grid = RandomizedSearchCV(pipeline, param_distributions=params, n_iter=5, n_jobs=-1)

grid.fit(X_train, y_train)
print("Best Parameters: ", grid.best_params_)
print("Accuracy Score on Test Data: {:.3f}".format(grid.score(X_test, y_test)))

Best Parameters:  {'rf_model__n_estimators': 200, 'rf_model__min_samples_leaf': 3, 'rf_model__max_depth': 5}
Accuracy Score on Test Data: 0.821


## Load UCI Income Dataset

In [ ]:
columns = ["age", "workclass", "fnlwgt", "education", "education_num", "maritial_status", "occupation", "relationship",
          "race", "sex", "capital_gain", "capital_loss", "", "native_country", "income"]

In [ ]:
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data", header=None, names=columns)

In [ ]:
df.info()

In [ ]:
features = ["age", "workclass", "education_num", "maritial_status", "occupation", "relationship",
          "race", "sex", "native_country", "income"]

X = df[features]
y = df["income"]

# Train / Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y)

X.sample(2)

## Put togehter a Pipeline

* We can Encode the Categories into Numbers (via Pandas - Map and Replace)
* Then put them in a Pipeline

OR

#### Just use SKLearn-Pandas

In [ ]:
mapper = DataFrameMapper([
                ("workclass", LabelBinarizer()),
                ("maritial_status", LabelBinarizer()),
                ("occupation", LabelBinarizer()),
                ("relationship", LabelBinarizer()),
                ("race", LabelBinarizer()),
                ("sex", LabelBinarizer()),
                ("native_country", LabelBinarizer()),
                ("workclass", LabelBinarizer()),
                ("workclass", LabelBinarizer()),
                (["age", "education_num"], None)  # No Transformations necessary
            ])

pipeline = Pipeline(steps=[
        ("features", mapper),
        ("model", LogisticRegression())
    ])

pipeline.fit(X_train, y_train)

print("Accuracy Score on Test Data: {:.3f}".format(pipeline.score(X_test, y_test)))

## Text Example - Ham vs Spam

* UCI Dataset - https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

In [ ]:
df = pd.read_csv("data/smsspamcollection/SMSSpamCollection", sep="\t", header=None, names=["target", "text"])

In [ ]:
X = df["text"]
y = df["target"]

In [ ]:
y.value_counts()

In [ ]:
# Convert Target to Int 0/1
y = (y == "spam").astype(int)

In [ ]:
# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn import metrics

In [ ]:
# Build a Model
log_reg_model = Pipeline(steps=[
        ("vect", CountVectorizer()), 
        ("model", LogisticRegression())
        ])

In [ ]:
log_reg_model.fit(X_train, y_train)

## Model Accuracy

In [ ]:
# Predictions
y_pred = log_reg_model.predict(X_test)

In [ ]:
# Accuracy
log_reg_model.score(X_test, y_test)

In [ ]:
# Confusion Matrix
metrics.confusion_matrix(y_test, y_pred)

## ROC / AUC

In [ ]:
y_pred_prob = log_reg_model.predict_proba(X_test)

In [ ]:
fpr, tpr, threshold = metrics.roc_curve(y_test, y_pred_prob[:, 1])

In [ ]:
roc_auc = metrics.auc(fpr, tpr)

In [ ]:
from matplotlib import pyplot as plt

%matplotlib inline

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange',
         lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right");

## Precision - Recall Curve

In [ ]:
### Precision / Recall
print(metrics.classification_report(y_test, y_pred))

In [ ]:
y_pred_prob = log_reg_model.predict_proba(X_test)

In [ ]:
precision, recall, threshold = metrics.precision_recall_curve(y_test, y_pred_prob[:, 1])

In [ ]:
# Plot precision-recall curve
plt.figure(figsize=(10, 6))
plt.plot(recall, precision, label='PR Curve', lw=2)
plt.plot(recall[1:],threshold, 'g--', lw=2, label="Threshold Prob") # threshold 
plt.legend(loc="lower left")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.xlim([0,1])
plt.ylim([0,1]);

### Scikit-Learn Pipelines make it easy to keep Feature Engineering and Model building together.  Use them.